In [3]:
#Please use python 3.5 or above
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input, Concatenate, Bidirectional, Conv2D, TimeDistributed, Add, Conv1D, Reshape, Flatten, Dropout, MaxPooling1D
from keras import optimizers
from keras.models import load_model, Model
import pandas as pd
import emoji
import json, argparse, os
import re
import io
import sys
from keras.utils import plot_model
import pydot
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Using TensorFlow backend.
W0429 12:25:55.234136 140736563217344 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [4]:
def is_emoji(s):
    return s in emoji.UNICODE_EMOJI

def add_space(text):
    return ''.join(' ' + char + ' ' if is_emoji(char) else char for char in text).strip()

def remove_text(text):
    words = text.split(' ')
    emojis = ' '.join([word for word in words if is_emoji(word)])
    return emojis

def count_length(text):
    return len(text)

def count_upper_case(text):
    return sum(1 for c in text if c.isupper())

def str2emoji(text):
    text = re.sub(r":‑\)"," ☺️ ",text)
    text = re.sub(r":\)\)\)\)"," ☺️ ",text)
    text = re.sub(r":\)\)\)"," ☺️ ",text)
    text = re.sub(r":\)\)"," ☺️ ",text)
    text = re.sub(r"\(:"," ☺️ ",text)
    text = re.sub(r":\)"," ☺️ ",text)
    text = re.sub(r":-\]"," ☺️ ",text)
    text = re.sub(r":\]"," ☺️ ",text)
    text = re.sub(r":-3"," ☺️ ",text)
    text = re.sub(r":3","  ☺️ ",text)
    text = re.sub(r":->"," ☺️ ",text)
    text = re.sub(r"</3",' 💔 ',text)
    text = re.sub(r"<3"," ❤️ ",text)
    text = re.sub(r":>"," ☺️ ",text)
    text = re.sub(r"8-\)"," ☺️ ",text)
    text = re.sub(r":o\)"," ☺️ ",text)
    text = re.sub(r":-\}"," ☺️ ",text)
    text = re.sub(r":\}"," ☺️ ",text)
    text = re.sub(r":-\)"," ☺️ ",text)
    text = re.sub(r":c\)"," ☺️ ",text)
    text = re.sub(r":\^\)"," ☺️ ",text)
    text = re.sub(r"=\]"," ☺️ ",text)
    text = re.sub(r"=\)"," ☺️ ",text)
    text = re.sub(r":‑D"," 😃 ",text)
    text = re.sub(r":D"," 😃 ",text)
    text = re.sub(r"8‑D"," 😃 ",text)
    text = re.sub(r"8D"," 😃 ",text)
    text = re.sub(r"X‑D"," 😃 ",text)
    text = re.sub(r"xD"," 😃 ",text)
    text = re.sub(r"xd"," 😃 ",text)
    text = re.sub(r"XD"," 😃 ",text)
    text = re.sub(r"=D"," 😃 ",text)
    text = re.sub(r"=3"," 😃 ",text)
    text = re.sub(r"B\^D"," 😃 ",text)
    text = re.sub(r":-\)\)"," 😃 ",text)
    text = re.sub(r":‑\("," ☹️ ",text)
    text = re.sub(r":-\("," ☹️ ",text)
    text = re.sub(r":-d", " 🤤 ",text)
    text = re.sub(r":d", " 🤤 ",text)
    text = re.sub(r"=d", " 🤤 ",text)

    text = re.sub(r":\("," ☹️ ",text)
    text = re.sub(r":‑c"," ☹️ ",text)
    text = re.sub(r":c"," ☹️ ",text)
    text = re.sub(r":‑<"," ☹️ ",text)
    text = re.sub(r":<"," ☹️ ",text)
    text = re.sub(r":‑\["," ☹️ ",text)
    text = re.sub(r":\["," ☹️ ",text)
    text = re.sub(r":-\|\|"," ☹️ ",text)
    text = re.sub(r">:\["," ☹️ ",text)
    text = re.sub(r":\{"," ☹️ ",text)
    text = re.sub(r":@"," ☹️ ",text)
    text = re.sub(r">:\("," ☹️ ",text)
    text = re.sub(r":'‑\("," 😭 ",text)
    text = re.sub(r":'\("," 😭 ",text)
    text = re.sub(r":'‑\)"," 😃 ",text)
    text = re.sub(r":'\)"," 😃 ",text)
    text = re.sub(r"D‑':"," 😧 ",text)
    text = re.sub(r"D:<"," 😨 ",text)
    text = re.sub(r"D:"," 😧 ",text)
    text = re.sub(r"D8"," 😧 ",text)
    text = re.sub(r"D;"," 😧 ",text)
    text = re.sub(r"D="," 😧 ",text)
    text = re.sub(r"DX"," 😧 ",text)
    text = re.sub(r":‑O"," 😮 ",text)
    text = re.sub(r":O"," 😮 ",text)
    text = re.sub(r":‑o"," 😮 ",text)
    text = re.sub(r":o"," 😮 ",text)
    text = re.sub(r":-0"," 😮 ",text)
    text = re.sub(r"8‑0"," 😮 ",text)
    text = re.sub(r">:O"," 😮 ",text)
    text = re.sub(r":-\*"," 😗 ",text)
    text = re.sub(r":\*"," 😗 ",text)
    text = re.sub(r":X"," 😗 ",text)
    text = re.sub(r";‑\)"," 😉 ",text)
    text = re.sub(r";\)"," 😉 ",text)
    text = re.sub(r"\*-\)"," 😉 ",text)
    text = re.sub(r"\*\)"," 😉 ",text)
    text = re.sub(r";‑\]"," 😉 ",text)
    text = re.sub(r";\]"," 😉 ",text)
    text = re.sub(r";\^\)"," 😉 ",text)
    text = re.sub(r":‑,"," 😉 ",text)
    text = re.sub(r";D"," 😉 ",text)
    text = re.sub(r":‑P"," 😛 ",text)
    text = re.sub(r":‑p"," 😛 ",text)
    text = re.sub(r":P"," 😛 ",text)
    text = re.sub(r":p"," 😛 ",text)
    text = re.sub(r"X‑P"," 😛 ",text)
    text = re.sub(r":‑Þ"," 😛 ",text)
    text = re.sub(r":Þ"," 😛 ",text)
    text = re.sub(r":b"," 😛 ",text)
    text = re.sub(r"d:"," 😛 ",text)
    text = re.sub(r"=p"," 😛 ",text)
    text = re.sub(r">:P"," 😛 ",text)
    text = re.sub(r":‑/"," 😕 ",text)
    text = re.sub(r":/"," 😕 ",text)
    text = re.sub(r":-\[\.\]"," 😕 ",text)
    text = re.sub(r">:/"," 😕 ",text)
    text = re.sub(r"=/"," 😕 ",text)
    text = re.sub(r":L"," 😕 ",text)
    text = re.sub(r"=L"," 😕 ",text)
    text = re.sub(r":S"," 😕 ",text)
    text = re.sub(r":‑\|"," 😐 ",text)
    text = re.sub(r":\|"," 😐 ",text)
    text = re.sub(r":$"," 😳 ",text)
    text = re.sub(r":‑x"," 🤐 ",text)
    text = re.sub(r":x"," 🤐 ",text)
    text = re.sub(r":‑#"," 🤐 ",text)
    text = re.sub(r":#"," 🤐 ",text)
    text = re.sub(r":‑&"," 🤐 ",text)
    text = re.sub(r":&"," 🤐 ",text)
    text = re.sub(r"O:‑\)"," 😇 ",text)
    text = re.sub(r"O:\)"," 😇 ",text)
    text = re.sub(r"0:‑3"," 😇 ",text)
    text = re.sub(r"0:3"," 😇 ",text)
    text = re.sub(r"0:‑\)"," 😇 ",text)
    text = re.sub(r"0:\)"," 😇 ",text)
    text = re.sub(r":‑b"," 😛 ",text)
    text = re.sub(r"0;\^\)"," 😇 ",text)
    text = re.sub(r">:‑\)"," 😈 ",text)
    text = re.sub(r">:\)"," 😈 ",text)
    text = re.sub(r"\}:‑\)"," 😈 ",text)
    text = re.sub(r"\}:\)"," 😈 ",text)
    text = re.sub(r"3:‑\)"," 😈 ",text)
    text = re.sub(r"3:\)"," 😈 ",text)
    text = re.sub(r">;\)"," 😈 ",text)
    text = re.sub(r"\|;‑\)"," 😎 ",text)
    text = re.sub(r"\|‑O"," 😏 ",text)
    text = re.sub(r":‑J"," 😏 ",text)
    text = re.sub(r"%‑\)"," 😵 ",text)
    text = re.sub(r"%\)"," 😵 ",text)
    text = re.sub(r":-###.."," 🤒 ",text)
    text = re.sub(r":###.."," 🤒 ",text)
    text = re.sub(r"\(>_<\)"," 😣 ",text)
    text = re.sub(r"\(>_<\)>"," 😣 ",text)
    text = re.sub(r"\(';'\)"," 👶 ",text)
    text = re.sub(r"\(\^\^>``"," 😓 ",text)
    text = re.sub(r"\(\^_\^;\)"," 😓 ",text)
    text = re.sub(r"\(-_-;\)"," 😓 ",text)
    text = re.sub(r"\(~_~;\) \(・\.・;\)"," 😓 ",text)
    text = re.sub(r"\(-_-\)zzz"," 😴 ",text)
    text = re.sub(r"\(\^_-\)"," 😉 ",text)
    text = re.sub(r"\(\(\+_\+\)\)"," 😕 ",text)
    text = re.sub(r"\(\+o\+\)"," 😕 ",text)
    text = re.sub(r"\^_\^"," 😃 ",text)
    text = re.sub(r"\(\^_\^\)/"," 😃 ",text)
    text = re.sub(r"\(\^O\^\)／"," 😃 ",text)
    text = re.sub(r"\(\^o\^\)／"," 😃 ",text)
    text = re.sub(r"\(__\)"," 🙇 ",text)
    text = re.sub(r"_\(\._\.\)_"," 🙇 ",text)
    text = re.sub(r"<\(_ _\)>"," 🙇 ",text)
    text = re.sub(r"<m\(__\)m>"," 🙇 ",text)
    text = re.sub(r"m\(__\)m"," 🙇 ",text)
    text = re.sub(r"m\(_ _\)m"," 🙇 ",text)
    text = re.sub(r"\('_'\)"," 😭 ",text)
    text = re.sub(r"\(/_;\)"," 😭 ",text)
    text = re.sub(r"\(T_T\) \(;_;\)"," 😭 ",text)
    text = re.sub(r"\(;_;"," 😭 ",text)
    text = re.sub(r"\(;_:\)"," 😭 ",text)
    text = re.sub(r"\(;O;\)"," 😭 ",text)
    text = re.sub(r"\(:_;\)"," 😭 ",text)
    text = re.sub(r"\(ToT\)"," 😭 ",text)
    text = re.sub(r";_;"," 😭 ",text)
    text = re.sub(r";-;"," 😭 ",text)
    text = re.sub(r";n;"," 😭 ",text)
    text = re.sub(r";;"," 😭 ",text)
    text = re.sub(r"Q\.Q"," 😭 ",text)
    text = re.sub(r"T\.T"," 😭 ",text)
    text = re.sub(r"QQ"," 😭 ",text)
    text = re.sub(r"Q_Q"," 😭 ",text)
    text = re.sub(r"\(-\.-\)"," 😞 ",text)
    text = re.sub(r"\(-_-\)"," 😞 ",text)
    text = re.sub(r"-_-"," 😞 ",text)
    text = re.sub(r"\(一一\)"," 😞 ",text)
    text = re.sub(r"\(；一_一\)"," 😞 ",text)
    text = re.sub(r"\(=_=\)"," 😩 ",text)
    text = re.sub(r"\(=\^\·\^=\)"," 😺 ",text)
    text = re.sub(r"\(=\^\·\·\^=\)"," 😺 ",text)
    text = re.sub(r"=_\^= "," 😺 ",text)
    text = re.sub(r"\(\.\.\)"," 😔 ",text)
    text = re.sub(r"\(\._\.\)"," 😔 ",text)
    text = re.sub(r"\(\・\・\?"," 😕 ",text)
    text = re.sub(r"\(\?_\?\)"," 😕 ",text)
    text = re.sub(r">\^_\^<"," 😃 ",text)
    text = re.sub(r"<\^!\^>"," 😃 ",text)
    text = re.sub(r"\^/\^"," 😃 ",text)
    text = re.sub(r"\（\*\^_\^\*）"," 😃 ",text)
    text = re.sub(r"\(\^<\^\) \(\^\.\^\)"," 😃 ",text)
    text = re.sub(r"\(^\^\)"," 😃 ",text)
    text = re.sub(r"\(\^\.\^\)"," 😃 ",text)
    text = re.sub(r"\(\^_\^\.\)"," 😃 ",text)
    text = re.sub(r"\(\^_\^\)"," 😃 ",text)
    text = re.sub(r"\(\^\^\)"," 😃 ",text)
    text = re.sub(r"\(\^J\^\)"," 😃 ",text)
    text = re.sub(r"\(\*\^\.\^\*\)"," 😃 ",text)
    text = re.sub(r"\(\^—\^\）"," 😃 ",text)
    text = re.sub(r"\(#\^\.\^#\)"," 😃 ",text)
    text = re.sub(r"\（\^—\^\）"," 👋 ",text)
    text = re.sub(r"\(;_;\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\^\.\^\)/~~~"," 👋 ",text)
    text = re.sub(r"\(T_T\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\*\^0\^\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*;"," 😍 ",text)
    text = re.sub(r"\(\+_\+\) \(@_@\)"," 😍 ",text)
    text = re.sub(r"\(\*\^\^\)v"," 😂 ",text)
    text = re.sub(r"\(\^_\^\)v"," 😂 ",text)
    text = re.sub(r"\(ーー;\)"," 😓 ",text)
    text = re.sub(r"\(\^0_0\^\)"," 😎 ",text)
    text = re.sub(r"\(\＾ｖ\＾\)"," 😀 ",text)
    text = re.sub(r"\(\＾ｕ\＾\)"," 😀 ",text)
    text = re.sub(r"\(\^\)o\(\^\)"," 😀 ",text)
    text = re.sub(r"\(\^O\^\)"," 😀 ",text)
    text = re.sub(r"\(\^o\^\)"," 😀 ",text)
    text = re.sub(r"\)\^o\^\("," 😀 ",text)
    text = re.sub(r":O o_O"," 😮 ",text)
    text = re.sub(r"o_0"," 😮 ",text)
    text = re.sub(r"o\.O"," 😮 ",text)
    text = re.sub(r"\(o\.o\)"," 😮 ",text)
    text = re.sub(r"oO"," 😮 ",text)
    text = re.sub(r':\‑\)','😃',text)
    text = re.sub(r":\)"," ☺️ ",text)
    text = re.sub(r":-]"," ☺️ ",text)
    text = re.sub(r":]"," ☺️ ",text)
    text = re.sub(r"8\-\)"," ☺️ ",text)
    text = re.sub(r":o\)"," ☺️ ",text)
    text = re.sub(r":-}"," ☺️ ",text)
    text = re.sub(r":}"," ☺️ ",text)
    text = re.sub(r":\-\)"," ☺️ ",text)
    text = re.sub(r":c\)"," ☺️ ",text)
    text = re.sub(r":^\)"," ☺️ ",text)
    text = re.sub(r"=]"," ☺️ ",text)
    text = re.sub(r"=\)"," ☺️ ",text)
    text = re.sub(r"B^D"," 😃 ",text)
    text = re.sub(r":-\)\)"," 😃 ",text)
    text = re.sub(r":-\("," ☹️ ",text)

    text = re.sub(r":‑\("," ☹️ ",text)
    text = re.sub(r":\("," ☹️ ",text)
    text = re.sub(r":\‑\["," ☹️ ",text)
    text = re.sub(r":\["," ☹️ ",text)
    text = re.sub(r":-\|\|"," ☹️ ",text)
    text = re.sub(r">\:\["," ☹️ ",text)
    text = re.sub(r":{"," ☹️ ",text)
    text = re.sub(r">\:\("," ☹️ ",text)
    text = re.sub(r":'‑\("," 😭 ",text)
    text = re.sub(r":'\("," 😭 ",text)
    text = re.sub(r":'\‑\)"," 😃 ",text)
    text = re.sub(r":'\)"," 😃 ",text)
    text = re.sub(r":-\*"," 😗 ",text)
    text = re.sub(r":\*"," 😗 ",text)
    text = re.sub(r";\‑\)"," 😉 ",text)
    text = re.sub(r";\)"," 😉 ",text)
    text = re.sub(r"\*\-\)"," 😉 ",text)
    text = re.sub(r"\*\)"," 😉 ",text)
    text = re.sub(r";‑\]"," 😉 ",text)
    text = re.sub(r";\]"," 😉 ",text)
    text = re.sub(r";^\)"," 😉 ",text)
    text = re.sub(r">\:\[\(\)\]"," 😕 ",text)

    text = re.sub(r":\[\(\)\]"," 😕 ",text)
    text = re.sub(r"=\[\(\)\]"," 😕 ",text)
    text = re.sub(r":‑\|"," 😐 ",text)
    text = re.sub(r":\|"," 😐 ",text)
    text = re.sub(r"O:‑\)"," 😇 ",text)
    text = re.sub(r"O:\)"," 😇 ",text)
    text = re.sub(r"0:‑\)"," 😇 ",text)
    text = re.sub(r"0:\)"," 😇 ",text)
    text = re.sub(r"0;^\)"," 😇 ",text)
    text = re.sub(r">:‑\)"," 😈 ",text)
    text = re.sub(r">:\)"," 😈 ",text)
    text = re.sub(r"}:‑\)"," 😈 ",text)
    text = re.sub(r"}:\)"," 😈 ",text)
    text = re.sub(r"3:‑\)"," 😈 ",text)
    text = re.sub(r"3:\)"," 😈 ",text)
    text = re.sub(r">;\)"," 😈 ",text)
    text = re.sub(r"\|;‑\)"," 😎 ",text)
    text = re.sub(r"\|‑O"," 😏 ",text)
    text = re.sub(r"%‑\)"," 😵 ",text)
    text = re.sub(r"%\)"," 😵 ",text)
    text = re.sub(r"\(>_<\)"," 😣 ",text)
    text = re.sub(r"\(>_<\)>"," 😣 ",text)
    text = re.sub(r"\(';'\)"," Baby ",text)
    text = re.sub(r"\(^^>``"," 😓 ",text)
    text = re.sub(r"\(^_^;\)"," 😓 ",text)
    text = re.sub(r"\(-_-;\)"," 😓 ",text)

    text = re.sub(r"\(~_~;\) \(・\.・;\)"," 😓 ",text)
    text = re.sub(r"\(-_-\)zzz"," 😴 ",text)
    text = re.sub(r"\(^_-\)"," 😉 ",text)
    text = re.sub(r"\(\(\+_\+\)\)"," 😕 ",text)
    text = re.sub(r"\(\+o\+\)"," 😕 ",text)
    text = re.sub(r"^_^"," 😃 ",text)
    text = re.sub(r"\(^_^\)/"," 😃 ",text)
    text = re.sub(r"\(^O^\)／"," 😃 ",text)
    text = re.sub(r"\(__\)"," 🙇 ",text)
    text = re.sub(r"_\(._.\)_"," 🙇 ",text)
    text = re.sub(r"<\(_ _\)>"," 🙇 ",text)
    text = re.sub(r"<m\(__\)m>"," 🙇 ",text)
    text = re.sub(r"m\(__\)m"," 🙇 ",text)
    text = re.sub(r"m\(_ _\)m"," 🙇 ",text)
    text = re.sub(r"\('_'\)"," 😭 ",text)
    text = re.sub(r"\(/_;\)"," 😭 ",text)
    text = re.sub(r"\(T_T\) (;_;)"," 😭 ",text)
    text = re.sub(r"\(;_;"," 😭 ",text)
    text = re.sub(r"\(;_:\)"," 😭 ",text)
    text = re.sub(r"\(;O;\)"," 😭 ",text)
    text = re.sub(r"\(:_;\)"," 😭 ",text)
    text = re.sub(r"\(ToT\)","  😭  ",text)
    text = re.sub(r"Q\.Q"," 😭 ",text)
    text = re.sub(r"T\.T"," 😭 ",text)
    text = re.sub(r"\(-\.-\)"," 😞 ",text)
    text = re.sub(r"\(-_-\)"," 😞 ",text)

    text = re.sub(r"\(一一\)"," 😞 ",text)
    text = re.sub(r"\(；一_一\)"," 😞 ",text)

    text = re.sub(r"\(=\_=\)"," 😩 ",text)
    text = re.sub(r"\(=^\·^=\)"," 😺 ",text)

    text = re.sub(r"\(=^··^=\)"," 😺 ",text)
    text = re.sub(r"=_^= "," 😺 ",text)

    text = re.sub(r"\(\.\.\)"," 😔 ",text)

    text = re.sub(r"\(\._\.\)"," 😔  ",text)
    text = re.sub(r"\(・・\?"," 😕 ",text)
    text = re.sub(r"\(\?_\?\)"," 😕 ",text)

    text = re.sub(r">^_^<"," 😃 ",text)
    text = re.sub(r"<^\!^>"," 😃 ",text)
    text = re.sub(r"^/^","  😃  ",text)
    text = re.sub(r"\(\*^_^\*\)","  😃  ",text)
    text = re.sub(r"\(^^\)","  😃  ",text)
    text = re.sub(r"\(^\.^\)","  😃  ",text)
    text = re.sub(r"\(^_^\.\)","  😃  ",text)
    text = re.sub(r"\(^_^\)","  😃  ",text)
    text = re.sub(r"\(^J^\)","  😃  ",text)
    text = re.sub(r"\(\*^\.^\*\)"," 😃  ",text)
    text = re.sub(r"\(^—^\）"," 😃 ",text)

    text = re.sub(r"\(#^.^#\)"," 😃 ",text)
    text = re.sub(r"\(^—^\)"," 👋 ",text)
    text = re.sub(r"\(;_;\)/~~~","  👋  ",text)

    text = re.sub(r"\(^.^\)/~~~"," 👋 ",text)
    text = re.sub(r"\(-_-\)/~~~ \($··\)/~~~"," 👋 ",text)
    text = re.sub(r"\(T_T\)/~~~"," 👋 ",text)

    text = re.sub(r"\(ToT\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\*^0^\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*\)"," 😍 ",text)

    text = re.sub(r"\(\*_\*;"," 😍 ",text)
    text = re.sub(r"\(+_+\) \(@_@\)"," 😍 ",text)
    text = re.sub(r"o\.O"," 😮 ",text)
    text = re.sub(r"\(o\.o\)"," 😮 ",text)

    return text

def preprocessString(text):
    repeatedChars = ['.', '?', '!', ',']
    for c in repeatedChars:
        lineSplit = text.split(c)
        while True:
            try:
                lineSplit.remove('')
            except:
                break
        cSpace = ' ' + c + ' '
        text = cSpace.join(lineSplit)
        
    text = fix_apos(text)
    
    return str(text)
    

def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in "train" mode] List of labels
    """
    indices = []
    conversations = []
    labels = []
    test_set = []
    u1 = []
    u2 = []
    u3 = []
    smil_1 = []
    smil_2 = []
    smil_3 = []
    smileys = []
    raw1 = []
    raw2 = []
    raw3 = []
    
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
            # Convert multiple instances of . ? ! , to single instance
            # okay...sure -> okay . sure
            # okay???sure -> okay ? sure
            # Add whitespace around such punctuation
            # okay!sure -> okay ! sure
            
            fix_emoji_line = str2emoji(line)
            splitted_line = fix_emoji_line.strip().split('\t')
            
            raw1.append(splitted_line[1])
            raw2.append(splitted_line[2])
            raw3.append(splitted_line[3]) 
            
            repeatedChars = ['.', '?', '!', ',']
            for c in repeatedChars:
                lineSplit = line.split(c)
                while True:
                    try:
                        lineSplit.remove('')
                    except:
                        break
                cSpace = ' ' + c + ' '
                line = cSpace.join(lineSplit)
                
            line = line.strip().split('\t')
                       
            if mode == "train":
                # Train data contains id, 3 turns and label
                label = emotion2label[line[4]]
                labels.append(label)
                test_set.append([line[4]])

            conv = ' <eos> '.join(line[1:4])
            
            #Replace non-unicode smilys with unicode
            conv = str2emoji(conv)
            
            #Separate smilys w unicode
            conv = add_space(conv)
            
            #Many of the words not in embeddings are problematic due to apostrophes e.g. didn't
            conv = fix_apos(conv)

            #Remove any duplicate spaces
            duplicateSpacePattern = re.compile(r'\ +')
            conv = re.sub(duplicateSpacePattern, ' ', conv)
            
            #Find all smilys as array of strings
            row_smileys = remove_text(conv)

            #Do the same operations for each turn
            u1_line = conv.split(' <eos> ')[0]
            u2_line = conv.split(' <eos> ')[1]
            u3_line = conv.split(' <eos> ')[2]
            
            #separate smilys per turn
            s1 = remove_text(u1_line)
            s2 = remove_text(u2_line)
            s3 = remove_text(u3_line)
            
            #remove double spaaces
            smil_1.append(re.sub(duplicateSpacePattern, ' ', s1))
            smil_2.append(re.sub(duplicateSpacePattern, ' ', s2))
            smil_3.append(re.sub(duplicateSpacePattern, ' ', s3))
                        
            u1.append(re.sub(duplicateSpacePattern, ' ', u1_line.lower()))
            u2.append(re.sub(duplicateSpacePattern, ' ', u2_line.lower()))
            u3.append(re.sub(duplicateSpacePattern, ' ', u3_line.lower()))
            smileys.append(row_smileys)            

            indices.append(int(line[0]))
            conversations.append(conv.lower())

    if mode == "train":
        return indices, conversations, labels, u1, u2, u3, smileys, smil_1, smil_2, smil_3, raw1, raw2, raw3
    else:
        return indices, conversations, u1, u2, u3, smileys
    
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with io.open(os.path.join('glove.6B', 'glove.6B.50d.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndex[word] = embeddingVector

    print('Found %s word vectors.' % len(embeddingsIndex))

    # Minimum word index of any word is 1.
    embeddingMatrix = np.zeros((len(wordIndex) + 1, 50))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector

    return embeddingMatrix

def find_converage(big_dict, small_dict):
    not_in_small = {}
    missing = 0
    exist_both = 0
    for key in small_dict:
        if key in big_dict:
            exist_both += 1
        else:
            not_in_small[key] = 0
            missing += 1
    coverage = exist_both/(exist_both + missing)
    return [coverage, missing, exist_both], not_in_small

def add_word(string, word):
    return 1 if word in string.split(' ') else 0

def find_number_missing(not_in_small, tokenizer_input_data):
    loop_count = 0
    for line in tokenizer_input_data:
        for key, _ in not_in_small.items():
            not_in_small[key] += add_word(line, key)
        loop_count += 1
        if not loop_count%10000:
            print("%s of %s"%(loop_count, len(tokenizer_input_data)))
    return not_in_small

def fix_apos(word):
    replaced_word = re.sub(r"(it['|´|’]s)", "it is", word)
    replaced_word = re.sub(r"(can['|´|’]t)", "can not", replaced_word)
    replaced_word = re.sub(r"(ain['|´|’]t)", "am not", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]re", r"\1 are", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]ll", r"\1 will", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]ve", r"\1 have", replaced_word)
    replaced_word = re.sub(r"(he|who|how|when|there)['|´|’]s", r"\1 is", replaced_word)
    replaced_word = re.sub(r"that['|´|’]s", r"that is", replaced_word)    
    replaced_word = re.sub(r"what['|´|’]s", r"what is", replaced_word)
    replaced_word = re.sub(r"(let['|´|’]s)", "let us", replaced_word)
    replaced_word = re.sub(r"([I|i]['|´|’]m)", "i am", replaced_word)
    replaced_word = re.sub(r"(won['|´|’]t)", "will not", replaced_word)
    replaced_word = re.sub(r"(n['|´|’]t)", " not", replaced_word)    
    replaced_word = re.sub(r"['|´|’]", r" ", replaced_word)
    return replaced_word

#separate words, then characters and pad as matrix eg:[[0,0,0,1,2,3],[1,2,3,4,5,6]] 
def padded_char_vectors(u1, u2, u3):
    tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK', lower=False)
    tk.fit_on_texts(u1+u2+u3)
    alphabet = " ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0gf123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    char_dict = {}
    t1 = {}
    t2 = {}
    t3 = {}
    word_list_t1 = []
    word_list_t2 = []
    word_list_t3 = []
    padded_seq_t1 = []
    padded_seq_t2 = []
    padded_seq_t3 = []
    
    #keras stuff for fixing dict
    for i, char in enumerate(alphabet):
        char_dict[char] = i + 1
    tk.word_index = char_dict.copy()
    tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
    
    #crazy stupid looping to get everything on the right shape
    #for text_to_sequences you want a list of lists of each letter per word per sentence
    for k, sentence in enumerate(u1):
        t1[k] = sentence.split(' ')
        t1[k] = [list(word) for word in t1[k]]
    for k, sentence in enumerate(u2):
        t2[k] = sentence.split(' ')
        t2[k] = [list(word) for word in t2[k]]
    for k, sentence in enumerate(u3):
        t3[k] = sentence.split(' ')
        t3[k] = [list(word) for word in t3[k]]
        
    #remaking to list of list        
    for key in t1:
        word_list_t1.append(t1[key])
    for key in t2:
        word_list_t2.append(t2[key])
    for key in t3:
        word_list_t3.append(t3[key])
    
    #pad the right things
    for word_per_turn in word_list_t1:
        seq_t1 = tk.texts_to_sequences(word_per_turn)
        padded1 = pad_sequences(seq_t1, maxlen=CHAR_MAX_LEN)
        padded_seq_t1.append(pad_array(padded1, CHAR_MAX_LEN))
        
    for word_per_turn in word_list_t2:
        seq_t2 = tk.texts_to_sequences(word_per_turn)
        padded2 = pad_sequences(seq_t2, maxlen=CHAR_MAX_LEN)
        padded_seq_t2.append(pad_array(padded2, CHAR_MAX_LEN))
    
    for word_per_turn in word_list_t3:
        seq_t3 = tk.texts_to_sequences(word_per_turn)
        padded3 = pad_sequences(seq_t3, maxlen=CHAR_MAX_LEN)
        padded_seq_t3.append(pad_array(padded3, CHAR_MAX_LEN))        
    
    return padded_seq_t1, padded_seq_t2, padded_seq_t3

#speparate characters and pad sentences, compared to above that uses matrices in which each row contains 
#character representation for each word in a sentence
def pad_sentence(u1, u2, u3):
    tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK', lower=False)
    tk.fit_on_texts(u1+u2+u3)
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0gf123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    char_dict = {}
    t1 = {}
    t2 = {}
    t3 = {}
    char_list_t1 = []
    char_list_t2 = []
    char_list_t3 = []
    padded_seq_t1 = []
    padded_seq_t2 = []
    padded_seq_t3 = []
    
    #keras stuff for fixing dict
    for i, char in enumerate(alphabet):
        char_dict[char] = i + 1
    tk.word_index = char_dict.copy()
    tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
    
    for sentence in u1:
        char_list_t1.append(list(sentence))
    char_list_t1 = tk.texts_to_sequences(char_list_t1)
    char_list_t1 = pad_sequences(char_list_t1, maxlen=CHAR_MAX_LEN)
    
    for sentence in u2:
        char_list_t2.append(list(sentence))
    char_list_t2 = tk.texts_to_sequences(char_list_t2)
    char_list_t2 = pad_sequences(char_list_t2, maxlen=CHAR_MAX_LEN)
    
    for sentence in u3:
        char_list_t3.append(list(sentence))
    char_list_t3 = tk.texts_to_sequences(char_list_t3)
    char_list_t3 = pad_sequences(char_list_t3, maxlen=CHAR_MAX_LEN)
    
    padded_seq_t1 = np.asarray(char_list_t1)
    padded_seq_t2 = np.asarray(char_list_t2)
    padded_seq_t3 = np.asarray(char_list_t3)
    
    return padded_seq_t1, padded_seq_t2, padded_seq_t3


def pad_array(matrix, pad_len):
    pad_diff = pad_len - matrix.shape[0]
    if pad_diff < 0:
        pad_diff = pad_len
    max_pad = np.zeros((pad_diff, matrix.shape[1]))
    padded_matrix = np.concatenate((matrix, max_pad), axis=0)
    padded_matrix = padded_matrix[0:pad_len,:]
    return padded_matrix
        
    

label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}



In [ ]:
indices, conversations, labels, v1, v2, v3, smileys, smil_v1, smil_v2, smil_v3, r1, r2, r3 = preprocessData('natemusMasters/starterkitdata/dev.txt', 'train')


In [ ]:
CHAR_MAX_LEN = 32
t1, t2, t3 = padded_char_vectors(r1, r2, r3)

In [ ]:
s1, s2, s3 = pad_sentence(r1, r2, r3)


In [28]:
def buildModel():
    char_input1 = Input(shape=(CHAR_MAX_LEN,))
    char_input2 = Input(shape=(CHAR_MAX_LEN,))
    char_input3 = Input(shape=(CHAR_MAX_LEN,))

    conv = Conv1D(kernel_size=4, filters=30, padding='same', activation='tanh', strides=1)
    lstm = Bidirectional(LSTM(units=256,return_sequences=False, recurrent_dropout=0.2))
    
    emb1 = Embedding(97, 32)(char_input1)
    emb2 = Embedding(97, 32)(char_input2)
    emb3 = Embedding(97, 32)(char_input3)
    
    conv1 = conv(emb1)
    conv2 = conv(emb2)
    conv3 = conv(emb3)
    
    conc_conv = Concatenate(axis=-1)([conv1, conv2, conv3])
    lstm_char = lstm(conc_conv)
    
    flat = Flatten()(conc_conv)
    
    model_output = Dense(4, activation='sigmoid')(flat)
    
    model = Model([char_input1, char_input2, char_input3], model_output)
    adam = optimizers.adam(lr=0.1)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])
    return model

model = buildModel()
model.summary()

NameError: name 'CHAR_MAX_LEN' is not defined

In [ ]:
stri = " ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0gf123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
len(stri)

In [ ]:
labels2 = to_categorical(np.asarray(labels))
m1 = model.fit([s1,s2,s3], labels2)

In [ ]:
df1 = pd.read_csv('predictions.csv')
df2 = pd.read_csv('data_w_predictions.csv')

df = pd.DataFrame({'conv': df1.Conversation, 'pred_model': df2.predictions, 'pred_BERT': df1.Predicted, 'true_label': df2.label})
df.pred_model = df.pred_model.apply(lambda x: emotion2label[x])
df.true_label = df.true_label.apply(lambda x: emotion2label[x])

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 200
number_diff = df.loc[df['pred_model'] != df['pred_BERT']].shape[0]
number_eq = df.loc[df['pred_model'] == df['pred_BERT']].shape[0]
bert_correct = df.loc[df['true_label'] == df['pred_BERT']].shape[0]
model_correct = df.loc[df['true_label'] == df['pred_model']].shape[0]
bert_not_model = df.loc[(df['true_label'] == df['pred_BERT']) & (df['true_label'] != df['pred_model'])].shape[0]
model_not_bert = df.loc[(df['true_label'] != df['pred_BERT']) & (df['true_label'] == df['pred_model'])].shape[0]
stats = {'number_diff': number_diff, 'number_equal': number_eq, 'bert_correct': bert_correct, 'model_correct': model_correct, 'bert_right_not_model': bert_not_model, 'model_right_not_bert': model_not_bert}



In [ ]:
bert_not_model

In [5]:
train = pd.read_csv('/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/train.txt', sep='\t')
dev = pd.read_csv('/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/dev.txt', sep='\t')
test = pd.read_csv('/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/testwithlabels.txt', sep='\t')
all_sets = [train, dev, test]

In [12]:
def other_or_emo(emotion):
    emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}
    return 0 if emotion2label[emotion]==0 else 1

def remove_emoji(text):
    return ''.join([char for char in list(text) if not is_emoji(char)]).strip()

train.label = train.label.apply(other_or_emo)
dev.label = dev.label.apply(other_or_emo)
test.label = test.label.apply(other_or_emo)


NameError: name 'train' is not defined

In [20]:
def preprocessData2(dir):
    df = pd.read_csv(dir, sep='\t')

    df.label = df.label.apply(other_or_emo)
    labels = df.label.tolist()

    df['t1_len'] = df['turn1'].apply(count_length)
    df['t2_len'] = df['turn2'].apply(count_length)
    df['t3_len'] = df['turn3'].apply(count_length)

    df['t1_upper'] = df['turn1'].apply(count_upper_case)
    df['t2_upper'] = df['turn2'].apply(count_upper_case)
    df['t3_upper'] = df['turn3'].apply(count_upper_case)
    
    df['t1_smil'] = df['turn1'].apply(str2emoji).apply(lambda x: sum([1 for s in x if is_emoji(s)]))
    df['t2_smil'] = df['turn2'].apply(str2emoji).apply(lambda x: sum([1 for s in x if is_emoji(s)]))
    df['t3_smil'] = df['turn3'].apply(str2emoji).apply(lambda x: sum([1 for s in x if is_emoji(s)]))
    
    df['turn1'] = df['turn1'].apply(str2emoji).apply(remove_emoji).apply(lambda x: x.lower()).apply(preprocessString)
    df['turn2'] = df['turn2'].apply(str2emoji).apply(remove_emoji).apply(lambda x: x.lower()).apply(preprocessString)
    df['turn3'] = df['turn3'].apply(str2emoji).apply(remove_emoji).apply(lambda x: x.lower()).apply(preprocessString)

    ind = df.id.tolist()

    t1, t2, t3 = np.asarray(df.turn1), np.asarray(df.turn2), np.asarray(df.turn3)
    len1, len2, len3 = np.asarray(df.t1_len), np.asarray(df.t2_len), np.asarray(df.t3_len)
    smil1, smil2, smil3 = np.asarray(df.t1_smil), np.asarray(df.t2_smil), np.asarray(df.t3_smil)
    case1, case2, case3 = np.asarray(df.t1_upper), np.asarray(df.t2_upper), np.asarray(df.t3_upper)
    
    return ind, labels, t1, t2, t3, len1, len2, len3, case1, case2, case3, smil1, smil2, smil3



In [23]:
validationDataPath = '/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/dev.txt'
testDataPath = '/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/testwithlabels.txt'
trainDataPath = '/Users/rasmushallen/Desktop/master_thesis_code/natemusMasters/starterkitdata/train.txt'
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100

ind, labels, t1, t2, t3, len1, len2, len3, case1, case2, case3, smil1, smil2, smil3 = preprocessData2(trainDataPath)
ind_val, validationLabels, t1_val, t2_val, t3_val, len1_val, len2_val, len3_val, case1_val, case2_val, case3_val, smil1_val, smil2_val, smil3_val = preprocessData2(validationDataPath)
ind_test, testLabels, t1_test, t2_test, t3_test, len1_test, len2_test, len3_test, case1_test, case2_test, case3_test, smil1_test, smil2_test, smil3_test = preprocessData2(testDataPath)


print("Extracting tokens...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(t1+t2+t3)
wordIndex = tokenizer.word_index

t1, t2, t3 = tokenizer.texts_to_sequences(t1), tokenizer.texts_to_sequences(t2), tokenizer.texts_to_sequences(t3)
t1_val, t2_val, t3_val = tokenizer.texts_to_sequences(t1_val), tokenizer.texts_to_sequences(t2_val), tokenizer.texts_to_sequences(t3_val)
t1_test, t2_test, t3_test = tokenizer.texts_to_sequences(t1_test), tokenizer.texts_to_sequences(t2_test), tokenizer.texts_to_sequences(t3_test)

print("Pad everything")
t1, t2, t3 = pad_sequences(t1, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t2, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t3, maxlen=MAX_SEQUENCE_LENGTH)
t1_val, t2_val, t3_val = pad_sequences(t1_val, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t2_val, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t3_val, maxlen=MAX_SEQUENCE_LENGTH)
t1_test, t2_test, t3_test = pad_sequences(t1_test, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t2_test, maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(t3_test, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
validationLabels = to_categorical(np.asarray(validationLabels))
testLabels = to_categorical(np.asarray(testLabels))

Extracting tokens...
Pad everything


In [24]:
np.random.shuffle(ind)
t1 = t1[ind]
t2 = t2[ind]
t3 = t3[ind]
case1 = np.asarray([case1[i] for i in ind])
case2 = np.asarray([case2[i] for i in ind])
case3 = np.asarray([case3[i] for i in ind])
len1 = np.asarray([len1[i] for i in ind])
len2 = np.asarray([len2[i] for i in ind])
len3 = np.asarray([len3[i] for i in ind])
smil1 = np.asarray([smil1[i] for i in ind])
smil2 = np.asarray([smil2[i] for i in ind])
smil3 = np.asarray([smil3[i] for i in ind])
labels = labels[ind]
metrics = {"accuracy" : [], "microPrecision" : [], "microRecall" : [], "microF1" : []}
embMat = np.zeros((20000, 200))


In [29]:
def buildModel(embeddingMatrix):
    """Constructs the architecture of the model
    Input:
        embeddingMatrix : The embedding matrix to be loaded in the embedding layer.
    Output:
        model : three layer lstm model with one layer of meta data
    """

    t1 = Input(shape=(100,), dtype='int32', name='turn_1')
    t2 = Input(shape=(100,), dtype='int32', name='turn_2')
    t3 = Input(shape=(100,), dtype='int32', name='turn_3')

    meta_data = Input(shape=(9,), dtype='float32', name='meta_input')
    
    ########## mata data layer #############
    hidden_layer = Dense(32, activation='relu')
    meta_layer = hidden_layer(meta_data)
    ########################################

    ########## Conversation layer ##########
    twitterEmbeddings = Embedding(embeddingMatrix.shape[0], EMBEDDING_DIM, weights=[embeddingMatrix], input_length=MAX_SEQUENCE_LENGTH, trainable=True)

    emb1 = twitterEmbeddings(t1)
    emb2 = twitterEmbeddings(t2)
    emb3 = twitterEmbeddings(t3)

    #LSTM layers, need to define a new one for different embeddings
    lstm = Bidirectional(LSTM(32, dropout=DROPOUT, return_sequences=True))
    second_layer = LSTM(64, dropout=DROPOUT)

    lstm1 = lstm(emb1)
    lstm2 = lstm(emb2)
    lstm3 = lstm(emb3)

    concatenated_lstm = Concatenate(axis=-1)([lstm1, lstm2, lstm3])
    concatenated_lstm = Dropout(DROPOUT)(concatenated_lstm)
    concatenated_lstm = second_layer(concatenated_lstm)
    concatenated_lstm = Dropout(DROPOUT)(concatenated_lstm)
    
    ############ Merging ############
    merge_out = Concatenate(axis=-1)([concatenated_lstm, meta_layer])

    #output
    model_output = Dense(2, activation='sigmoid')(merge_out)
    model_output = Dense()
    model = Model([t1, t2, t3, meta_data], model_output)

    rmsprop = optimizers.rmsprop(lr=LEARNING_RATE)
    adam =  optimizers.adam(lr=LEARNING_RATE)

    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])

    return model

In [30]:
print("Set up mo.. mod... mod..EL.. MODEL MOOODEEEL")

NUM_EPOCHS=30
LEARNING_RATE=0.005
LSTM_DIM = 128
BATCH_SIZE = 200
EMBEDDING_DIM = 200
DROPOUT = 0.3

meta_data = np.asarray([len1, len2, len3, case1, case2, case3, smil1, smil2, smil3]).T
meta_data_val = np.asarray([len1_val, len2_val, len3_val, case1_val, case2_val, case3_val, smil1_val, smil2_val, smil3_val]).T
meta_data_test = np.asarray([len1_test, len2_test, len3_test, case1_test, case2_test, case3_test, smil1_test, smil2_test, smil3_test]).T

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)
mc = ModelCheckpoint('EP%d_LR%de-5_LDim%d_BS%d.h5'%(NUM_EPOCHS, int(LEARNING_RATE*(10**5)), LSTM_DIM, BATCH_SIZE), monitor='val_loss', mode='min', verbose=1, save_best_only=True)
model = buildModel(embMat)
model.summary()
history = model.fit([t1, t2, t3, meta_data], labels, validation_data=([t1_val, t2_val, t3_val, meta_data_val], validationLabels), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, verbose=2, callbacks=[es, mc])


Set up mo.. mod... mod..EL.. MODEL MOOODEEEL
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
turn_1 (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
turn_2 (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
turn_3 (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 100, 200)     4000000     turn_1[0][0]                     
                                                                

KeyboardInterrupt: 